# State Design Pattern

In [1]:
class IState{
    public:
    virtual ~IState() = default;
    virtual void insertQuarter() = 0;
    virtual void ejectQuarter() = 0;
    virtual void turnCrank() = 0;
    virtual void dispense() = 0;
};

In [2]:
class IStateMachine
{
    public:
    IStateMachine(IState* state):m_state(state) {};
    virtual ~IStateMachine() = default;
    virtual void insertQuarter() = 0;
    virtual void ejectQuarter()= 0;
    virtual void turnCrank()= 0;
    virtual void setState(IState* state) = 0;
    virtual void releaseBall() = 0;
    virtual int getCount() = 0;
    virtual IState* getNoQuarterState() = 0;
    virtual IState* getSoldOutState() = 0;
    virtual IState* getHasQuarterState() = 0;
    virtual IState* getSoldState() = 0;
    virtual IState* getWinnerState() = 0;
    
    protected:
    IState* soldOutState;
    IState* noQuarterState;
    IState* hasQuarterState;
    IState* soldState;
    IState* winnerState; 
    IState* m_state;
};

In [3]:
#include <iostream>
//class CGumballMachine;

class CSoldState: public IState
{
public:
CSoldState(IStateMachine& gumballMachine):m_GumballMachine(gumballMachine){}
virtual ~CSoldState() = default;
virtual void insertQuarter() override final {std::cout << "Please wait, we are already giving you a gumball\n";}
virtual void ejectQuarter() override final{std::cout << "You already turned the crank\n";}
virtual void turnCrank() override final{std::cout << "Turning twice does't get another ball\n";}
virtual void dispense() override final{
    m_GumballMachine.releaseBall();
    if (m_GumballMachine.getCount() > 0)
    {
        m_GumballMachine.setState(m_GumballMachine.getNoQuarterState());
    }
    else
    {
        std::cout << "Sold out...\n";
        m_GumballMachine.setState(m_GumballMachine.getSoldOutState());
    }
}
private:
IStateMachine& m_GumballMachine;
    
};

In [4]:
class CSoldOutState: public IState
{
public:
CSoldOutState(IStateMachine& gumballMachine):m_GumballMachine(gumballMachine){}
virtual ~CSoldOutState() = default;
virtual void insertQuarter() override final {}
virtual void ejectQuarter() override final{}
virtual void turnCrank() override final{}
virtual void dispense() override final{}

private:
IStateMachine& m_GumballMachine;
    
};

In [5]:
class CNoQuarterState: public IState
{
public:
CNoQuarterState(IStateMachine& gumballMachine):m_GumballMachine(gumballMachine){}
virtual ~CNoQuarterState() = default;
virtual void insertQuarter() override final 
{
    std::cout << "You inserted a quarter\n";
    m_GumballMachine.setState(m_GumballMachine.getHasQuarterState());
    
}
virtual void ejectQuarter() override final
{
 std::cout << "You havent inserted a quarter\n";
}
virtual void turnCrank() override final
{
 std::cout << "You turned but there is no quarter\n";
}
virtual void dispense() override final
{
 std::cout << "You need to pay first\n";
}
    
private:
IStateMachine& m_GumballMachine;
    
};

In [6]:
class CHasQuarterState: public IState
{
public:
CHasQuarterState(IStateMachine& gumballMachine):m_GumballMachine(gumballMachine) {}
virtual ~CHasQuarterState() = default;
virtual void insertQuarter() override final {std::cout << "You can't insert another quarter\n";}
virtual void ejectQuarter() override final
{
    std::cout << "Quarter returned\n";
    m_GumballMachine.setState(m_GumballMachine.getNoQuarterState());
}
virtual void turnCrank() override final
{
    std::cout << "You turned...\n";
    m_Secret = rand() % 10;
    if((m_Secret == 0) && m_GumballMachine.getCount() > 1)
    {
        m_GumballMachine.setState(m_GumballMachine.getWinnerState());
    }
    else
    {
        m_GumballMachine.setState(m_GumballMachine.getSoldState());
    }
    
}
virtual void dispense() override final{std::cout << "No gumball dispensed\n";}
    
private:
IStateMachine& m_GumballMachine;
int m_Secret;
};

In [7]:
class CWinnerState: public IState
{
public:
CWinnerState(IStateMachine& gumballMachine):m_GumballMachine(gumballMachine) {}
virtual ~CWinnerState() = default;
virtual void insertQuarter() override final {std::cout << "Error:insertQuarter in winner state\n";}
virtual void ejectQuarter() override final{std::cout << "Error ejectQuarter in winner state\n";}
virtual void turnCrank() override final{std::cout << "Error turnCrank in winner state\n";}
virtual void dispense() override final
{
    std::cout << "You are a winner\n";
    m_GumballMachine.releaseBall();
    if (m_GumballMachine.getCount() == 0)
    {
        m_GumballMachine.setState(m_GumballMachine.getSoldOutState());
    }
    else
    {
        m_GumballMachine.releaseBall();
        if (m_GumballMachine.getCount() > 0)
        {
            m_GumballMachine.setState(m_GumballMachine.getNoQuarterState());
        }
        else
        {
            std::cout << "out of ball...\n";
            m_GumballMachine.setState(m_GumballMachine.getSoldOutState());
        }
    }    
}
private:
IStateMachine& m_GumballMachine;    
};

In [8]:
class CGumballMachine: public IStateMachine
{
    public:
    CGumballMachine(int numberGumballs):IStateMachine(nullptr),m_count(numberGumballs)
    {
        soldOutState = new CSoldOutState(*this);
        noQuarterState = new CNoQuarterState(*this);
        hasQuarterState = new CHasQuarterState(*this);
        soldState = new CSoldState(*this);
        winnerState = new CWinnerState(*this);
        m_state = soldOutState;
        if (m_count > 0)
        {
            m_state = noQuarterState;
        }
    }
    
    virtual ~CGumballMachine() = default;
    
    virtual void insertQuarter() override final
    {
        m_state->insertQuarter();
    }
    
    virtual void ejectQuarter() override final
    {
        m_state->ejectQuarter();
    }
    
    virtual void turnCrank() override final
    {
        m_state->turnCrank();
        m_state->dispense();
    }
    
    virtual void setState(IState* state) override final
    {
        m_state = state;
    }
    
    virtual void releaseBall() override final
    {
        std::cout << "A gumball comes rolling out the slot...\n";
        if (m_count !=0)
        {
            m_count = m_count - 1;
            std::cout << m_count <<"\n";
        }
    }
    
    virtual int getCount() override final
    {
        return m_count;
    }
    
    virtual IState* getSoldOutState() override final {return soldOutState;}
    virtual IState* getNoQuarterState() override final {return noQuarterState;}
    virtual IState* getHasQuarterState() override final {return hasQuarterState;}
    virtual IState* getSoldState() override final {return soldState;}
    virtual IState* getWinnerState() override final {return winnerState;}
    
    int m_count = 0;
};

In [9]:
 
    CGumballMachine* gumballMachine = new CGumballMachine(4);
   
    gumballMachine->insertQuarter();
    gumballMachine->turnCrank();
    
    gumballMachine->insertQuarter();
    gumballMachine->turnCrank();
    
    gumballMachine->insertQuarter();
    gumballMachine->turnCrank();


You inserted a quarter
You turned...
A gumball comes rolling out the slot...
3
You inserted a quarter
You turned...
You are a winner
A gumball comes rolling out the slot...
2
A gumball comes rolling out the slot...
1
You inserted a quarter
You turned...
A gumball comes rolling out the slot...
0
Sold out...
